In [1]:
#cleaning the data

In [9]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime


In [10]:
senators_file= "1976-2018-senate.csv"

In [11]:
# new_senate=pd.read_csv(senators_file)

new_senate = pd.read_csv(senators_file, encoding= 'unicode_escape')


In [12]:
df= pd.DataFrame(new_senate)
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Wm. Mathews Feighan,independent,False,total,1565,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Allan Norwitz,libertarian,False,total,7310,741210,False,20171011.0
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Bob Field,independent,False,total,10765,741210,False,20171011.0


In [13]:
datayear = df[(df.year>=2014) & (df.year<=2018)]
datayear

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
2861,2014,Alabama,AL,1,63,41,US Senate,statewide,gen,False,NaN,NaN,True,total,22484,818090,False,20171011.0
2862,2014,Alabama,AL,1,63,41,US Senate,statewide,gen,False,Jeff Sessions,republican,False,total,795606,818090,False,20171011.0
2863,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,NaN,NaN,True,total,1376,282400,False,20171011.0
2864,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,Mark Begich,democrat,False,total,129431,282400,False,20171011.0
2865,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,Mark S. Fish,libertarian,False,total,10512,282400,False,20171011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,55,35,25,US Senate,statewide,gen,False,NaN,NaN,True,total,42,2657841,False,20190110.0
3417,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,John Barrasso,republican,False,total,136210,203420,False,20190110.0
3418,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,Gary Trauner,democrat,False,total,61227,203420,False,20190110.0
3419,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,Joseph Porambo,libertarian,False,total,5658,203420,False,20190110.0


In [14]:
new_df=datayear[["year","state","state_po","candidate","party","candidatevotes","writein","totalvotes"]]
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,NaN,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,NaN,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,NaN,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [15]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 2861 to 3420
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            560 non-null    int64 
 1   state           560 non-null    object
 2   state_po        560 non-null    object
 3   candidate       468 non-null    object
 4   party           439 non-null    object
 5   candidatevotes  560 non-null    int64 
 6   writein         560 non-null    bool  
 7   totalvotes      560 non-null    int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 35.5+ KB


In [16]:
new_df.columns

Index(['year', 'state', 'state_po', 'candidate', 'party', 'candidatevotes',
       'writein', 'totalvotes'],
      dtype='object')

In [17]:
new_df['candidate'] = new_df['candidate'].replace(np.NaN, "Write in")
new_df


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [18]:
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [19]:
new_df['party'] = new_df['party'].replace(np.NaN, "No Party")
new_df


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [20]:
new_df["party"]= new_df["party"].replace(["republican"], "Republican")
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [21]:
new_df["party"]= new_df["party"].replace(["democrat"], "Democrat")
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [22]:
new_df["party"]= new_df["party"].replace(["libertarian"], "Libertarian")
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,Libertarian,5658,False,203420


In [23]:
new_df.shape

(560, 8)

In [24]:
# new_df.to_csv('votersinfo.csv', index = False)

In [25]:
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,Libertarian,5658,False,203420


In [26]:
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    #print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [27]:
def whichState(states):
    elections = []
    for i in states:
        if i not in elections:
            elections.append(i)
    return elections

In [28]:
votersInfo2014 = new_df[(new_df.year == 2014)].reset_index()
votersInfo2014.head()

,index,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
0,2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
1,2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2,2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
3,2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
4,2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400


In [29]:
votersInfo2014 = votersInfo2014.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2014 = votersInfo2014.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2014))
#votersInfo2014[:10]

187


In [30]:
#Splitting Dataframe into two major parties
smallerList = ['Repub']
longerList = votersInfo2014['Party']
print('Republicans')
index2014 = matchLists(longerList, smallerList)
repubParty2014 = votersInfo2014.iloc[index2014, :]
repubParty2014 = repubParty2014.reset_index()
repubParty2014 = repubParty2014.drop(['index'], axis = 1)
##
smallerList = ['Demo']
longerList = votersInfo2014['Party']
print('Democrats')
index2014 = matchLists(longerList, smallerList)
democParty2014 = votersInfo2014.iloc[index2014, :]
democParty2014 = democParty2014.reset_index()
democParty2014 = democParty2014.drop(['index'], axis = 1)

Republicans
length uniques only: 38
Democrats
length uniques only: 37


In [31]:
repubParty2014.head(5)

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Alabama,AL,Jeff Sessions,Republican,795606,818090
1,Alaska,AK,Dan Sullivan,Republican,135445,282400
2,Arkansas,AR,Tom Cotton,Republican,478819,847505
3,Colorado,CO,Cory Gardner,Republican,983891,2041058
4,Delaware,DE,Kevin Wade,Republican,98823,234038


In [32]:
democParty2014.head(5)

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Alaska,AK,Mark Begich,Democrat,129431,282400
1,Arkansas,AR,Mark L. Pryor,Democrat,334174,847505
2,Colorado,CO,Mark Udall,Democrat,944203,2041058
3,Delaware,DE,Christopher A. Coons,Democrat,130655,234038
4,Georgia,GA,M. Michelle Nunn,Democrat,1160811,2567805


In [33]:
repubParty2014.to_csv('repubParty2014.csv', index = False)
democParty2014.to_csv('democParty2014.csv', index = False)

In [34]:
# smallerList = ['Demo']
# longerList = votersInfo2014['Party']
# index2014 = matchLists(longerList, smallerList)
# democParty2014 = votersInfo2014.iloc[index2014, :]
# democParty2014 = democParty2014.reset_index()
# democParty2014 = democParty2014.drop(['index'], axis = 1)
# democParty2014.head(5)

In [35]:
#which states elected in 2014
stateList = list(votersInfo2014['State'])
election2014 = whichState(votersInfo2014['State'])
print(election2014)
len(election2014)

['Alabama', 'Alaska', 'Arkansas', 'Colorado', 'Delaware', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Montana', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'North Carolina', 'Oklahoma', 'Oregon', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Virginia', 'West Virginia', 'Wyoming']


34

In [36]:
votersInfo2016 = new_df[(new_df.year == 2016)].reset_index()
votersInfo2016.head()

,index,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
0,3048,2016,Alabama,AL,Ron Crumpton,Democrat,748709,False,2087444
1,3049,2016,Alabama,AL,Richard C. Shelby,Republican,1335104,False,2087444
2,3050,2016,Alabama,AL,Write in,No Party,3631,True,2087444
3,3051,2016,Alaska,AK,Joe Miller,Libertarian,90825,False,311441
4,3052,2016,Alaska,AK,Ted Gianoutsos,independent,1758,False,311441


In [37]:
votersInfo2016 = votersInfo2016.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2016 = votersInfo2016.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2016))
votersInfo2016[:10]

221


,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Alabama,AL,Ron Crumpton,Democrat,748709,2087444
1,Alabama,AL,Richard C. Shelby,Republican,1335104,2087444
2,Alabama,AL,Write in,No Party,3631,2087444
3,Alaska,AK,Joe Miller,Libertarian,90825,311441
4,Alaska,AK,Ted Gianoutsos,independent,1758,311441
5,Alaska,AK,Ray Metcalfe,Democrat,36200,311441
6,Alaska,AK,Margaret Stock,independent,41194,311441
7,Alaska,AK,Breck A. Craig,independent,2609,311441
8,Alaska,AK,Lisa Murkowski,Republican,138149,311441
9,Alaska,AK,Write in,No Party,706,311441


In [38]:
#which states elected in 2016
stateList = list(votersInfo2016['State'])
election2016 = whichState(votersInfo2016['State'])
print(election2016)
len(election2016)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maryland', 'Missouri', 'Nevada', 'New Hampshire', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'South Carolina', 'South Dakota', 'Utah', 'Vermont', 'Washington', 'Wisconsin']


34

In [39]:
#Splitting Dataframe into two major parties
smallerList = ['Repub']
longerList = votersInfo2016['Party']
print('Republicans')
index2016 = matchLists(longerList, smallerList)
repubParty2016 = votersInfo2016.iloc[index2016, :]
repubParty2016 = repubParty2016.reset_index()
repubParty2016 = repubParty2016.drop(['index'], axis = 1)
##
smallerList = ['Demo']
longerList = votersInfo2016['Party']
print('Democrats')
index2016 = matchLists(longerList, smallerList)
democParty2016 = votersInfo2016.iloc[index2016, :]
democParty2016 = democParty2016.reset_index()
democParty2016 = democParty2016.drop(['index'], axis = 1)

Republicans
length uniques only: 42
Democrats
length uniques only: 41


In [40]:
repubParty2016.head()

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Alabama,AL,Richard C. Shelby,Republican,1335104,2087444
1,Alaska,AK,Lisa Murkowski,Republican,138149,311441
2,Arizona,AZ,Sydney Dudikoff,Republican,494,2530730
3,Arizona,AZ,John McCain,Republican,1359267,2530730
4,Arkansas,AR,John Boozman,Republican,661984,1107522


In [41]:
democParty2016.head()

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Alabama,AL,Ron Crumpton,Democrat,748709,2087444
1,Alaska,AK,Ray Metcalfe,Democrat,36200,311441
2,Arizona,AZ,Ann Kirkpatrick,Democrat,1031245,2530730
3,Arkansas,AR,Conner Eldridge,Democrat,400602,1107522
4,California,CA,Loretta Sanchez,Democrat,4701417,12244170


In [42]:
repubParty2016.to_csv('repubParty2016.csv', index = False)
democParty2016.to_csv('democParty2016.csv', index = False)

In [43]:
votersInfo2018 = new_df[(new_df.year == 2018)].reset_index()
#votersInfo2018.head()

In [44]:
votersInfo2018 = votersInfo2018.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2018 = votersInfo2018.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2018))
votersInfo2018[:10]

152


,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Arizona,AZ,Martha McSally,Republican,1135200,2384308
1,Arizona,AZ,Kyrsten Sinema,Democrat,1191100,2384308
2,Arizona,AZ,Angela Green,green,57442,2384308
3,Arizona,AZ,Write in,No Party,566,2384308
4,California,CA,Dianne Feinstein,Democrat,6019422,11113364
5,California,CA,Kevin De Leon,Democrat,5093942,11113364
6,Connecticut,CT,Christopher S Murphy,Democrat,787685,1386840
7,Connecticut,CT,Matthew Corey,Republican,545717,1386840
8,Connecticut,CT,Christopher S Murphy,working families,37894,1386840
9,Connecticut,CT,Richard Lion,Libertarian,8838,1386840


In [45]:
#which states elected in 2018
stateList = list(votersInfo2018['State'])
election2018 = whichState(votersInfo2018['State'])
print(election2018)
len(election2018)

['Arizona', 'California', 'Connecticut', 'Delaware', 'Florida', 'Hawaii', 'Indiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Jersey', 'New Mexico', 'New York', 'North Dakota', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


33

In [46]:
#Splitting Dataframe into two major parties
smallerList = ['Repub']
longerList = votersInfo2018['Party']
print('Republicans')
index2018 = matchLists(longerList, smallerList)
repubParty2018 = votersInfo2018.iloc[index2018, :]
repubParty2018 = repubParty2018.reset_index()
repubParty2018 = repubParty2018.drop(['index'], axis = 1)
##
smallerList = ['Demo']
longerList = votersInfo2018['Party']
print('Democrats')
index2018 = matchLists(longerList, smallerList)
democParty2018 = votersInfo2018.iloc[index2018, :]
democParty2018 = democParty2018.reset_index()
democParty2018 = democParty2018.drop(['index'], axis = 1)

Republicans
length uniques only: 34
Democrats
length uniques only: 32


In [47]:
repubParty2018.head()

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Arizona,AZ,Martha McSally,Republican,1135200,2384308
1,Connecticut,CT,Matthew Corey,Republican,545717,1386840
2,Delaware,DE,Robert B Arlett,Republican,137127,362592
3,Florida,FL,Rick Scott,Republican,4099505,8190005
4,Hawaii,HI,Ron Curtis,Republican,112035,388351


In [48]:
democParty2018.head()

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Arizona,AZ,Kyrsten Sinema,Democrat,1191100,2384308
1,California,CA,Dianne Feinstein,Democrat,6019422,11113364
2,California,CA,Kevin De Leon,Democrat,5093942,11113364
3,Connecticut,CT,Christopher S Murphy,Democrat,787685,1386840
4,Delaware,DE,Thomas R Carper,Democrat,217385,362592


In [49]:
repubParty2018.to_csv('repubParty2018.csv', index = False)
democParty2018.to_csv('democParty2018.csv', index = False)

# CSVs 4 & 5
- Bringing in Chair_Members to match with votersInfo
- Bringing in Ranking_Members to match with votersInfo



In [50]:
#Bringing in Chair_Members to match with votersInfo
chairs_file= "Chair_Members.csv"
matchingChairs = pd.read_csv(chairs_file, encoding= 'unicode_escape')

In [51]:
matchingChairs.head()

,Committee,SubCommittee,C_First_Name,C_Last_Name,C_Party,State_Abbrv,Chair_Names,State,C_Party_Name
0,Committee,"Agriculture, Nutrition and Forestry",Pat,Roberts,R,MI,Pat Roberts,Michigan,Republican
1,SubCommittee,"Commodities, Risk Management and Trade",John,Boozman,R,OH,John Boozman,Ohio,Republican
2,SubCommittee,"Conservation, Forestry and Natural Resources",Mike,Braun,R,CO,Mike Braun,Colorado,Republican
3,SubCommittee,"Livestock, Marketing and Agriculture Security",Cindy,Hyde-Smith,R,NY,Cindy Hyde-Smith,New York,Republican
4,SubCommittee,"Nutrition, Agricultural Research and Specialty...",Deb,Fischer,R,PA,Deb Fischer,Pennsylvania,Republican


In [52]:
matchingChairs = matchingChairs[(matchingChairs.Committee == "Committee")].reset_index()
matchingChairs = matchingChairs.drop(['Committee', "C_Party", "C_Party_Name", "index"], axis = 1)
matchingChairs = matchingChairs.rename(columns= {'SubCommittee':'Committee'})

In [53]:
len(matchingChairs)

16

In [54]:
matchingChairs

,Committee,C_First_Name,C_Last_Name,State_Abbrv,Chair_Names,State
0,"Agriculture, Nutrition and Forestry",Pat,Roberts,MI,Pat Roberts,Michigan
1,Appropriations,Richard,Shelby,VT,Richard Shelby,Vermont
2,Armed Services,Jim,Inhofe,RI,Jim Inhofe,Rhode Island
3,"Banking, Housing and Urban Affairs",Mike,Crapo,OH,Mike Crapo,Ohio
4,Budget,Mike,Enzi,VT,Mike Enzi,Vermont
5,"Commerce, Science and Transportation",Roger,Wicker,WA,Roger Wicker,Washington
6,Energy and Natural Resources,Lisa,Murkowski,WV,Lisa Murkowski,West Virginia
7,Environment and Public Works,John,Barrasso,DE,John Barrasso,Delaware
8,Finance,Chuck,Grassley,OR,Chuck Grassley,Oregon
9,Foreign Relations,Jim,Risch,NJ,Jim Risch,New Jersey


In [55]:
#Bringing in Ranking_Members to match with votersInfo
ranking_file= "Ranking_Members.csv"
matchingRanking = pd.read_csv(ranking_file, encoding= 'unicode_escape')

In [56]:
matchingRanking = matchingRanking[(matchingRanking.Committee == "Committee")].reset_index()
matchingRanking = matchingRanking.drop(['Committee', "RM_Party", "R_Party_Name", "index"], axis = 1)
matchingRanking = matchingRanking.rename(columns= {'SubCommittee':'Committee'})

In [57]:
len(matchingRanking)

16

In [58]:
matchingRanking

,Committee,R_First_Name,R_Last_Name,State_Abbrv,Ranking_Members,State
0,"Agriculture, Nutrition and Forestry",Debbie,Stabenow,MI,Debbie Stabenow,Michigan
1,Appropriations,Patrick,Leahy,VT,Patrick Leahy,Vermont
2,Armed Services,Jack,Reed,RI,Jack Reed,Rhode Island
3,"Banking, Housing and Urban Affairs",Sherrod,Brown,OH,Sherrod Brown,Ohio
4,Budget,Bernie,Sanders,VT,Bernie Sanders,Vermont
5,"Commerce, Science and Transportation",Maria,Cantwell,WA,Maria Cantwell,Washington
6,Energy and Natural Resources,Joe,Manchin,WV,Joe Manchin,West Virginia
7,Environment and Public Works,Tom,Carper,DE,Tom Carper,Delaware
8,Finance,Ron,Wyden,OR,Ron Wyden,Oregon
9,Foreign Relations,Bob,Menendez,NJ,Bob Menendez,New Jersey


# Cleaning/Removing dups 2014
- votersinfo2018 has duplicate rows of a few names, searched for names and added their total votes
- this probably occured because of how the party was entered

In [59]:
votersInfo2014['Candidate'][38] = 'James D. Oberweis'
votersInfo2014['Candidate'][68] = 'Other'
votersInfo2014['Candidate'][164] = "Emily Sanchez"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [60]:
#votersInfo2014.iloc[90:150]

# Cleaning/Removing dups 2016
- votersinfo2018 has duplicate rows of a few names, searched for names and added their total votes
- this probably occured because of how the party was entered

In [61]:
votersInfo2016['Candidate'][97] = "M. V. Mendoza"
votersInfo2016['Candidate'][95] = "Charles W. Boustany Jr."
votersInfo2016['Candidate'][102] = "Rob Maness"
votersInfo2016['Candidate'][105] = "William Robert LangJr"
votersInfo2016['Candidate'][110] = "Gregory TaylorJr"
votersInfo2016['Candidate'][115] = "Joseph Cao"
votersInfo2016['Candidate'][117] = "Donald Crawford"
votersInfo2016['Candidate'][140] = "Thomas Sawyer"
votersInfo2016['Candidate'][144] = "Other"
votersInfo2016['Candidate'][154] = "Other"
votersInfo2016['Candidate'][155] = "Other"
votersInfo2016['Candidate_Votes'][34] = 920766+87948
votersInfo2016['Candidate'][56] = "Other"
votersInfo2016['Candidate'][57] = "John M. Giuffre"
votersInfo2016['Candidate'][61] = "Other"
votersInfo2016['Candidate_Votes'][151] = 45402+4784220+150655
votersInfo2016['Candidate_Votes'][152] = 267613+1723927+17813
votersInfo2016['Candidate'][205] = "Other"


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [62]:
#votersInfo2016.iloc[200:220]

In [63]:
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[35])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[156])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[158])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[161])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[155])
votersInfo2016 = votersInfo2016.drop(votersInfo2016.index[159])
votersInfo2016 = votersInfo2016.reset_index()
votersInfo2016 = votersInfo2016.drop(['index'], axis = 1)

# Cleaning/Removing dups 2018
- votersinfo2018 has duplicate rows of a few names, searched for names and added their total votes
- this probably occured because of how the party was entered

In [64]:
#Found dups 2018
# Kirsten E. Gillibrand
# Chele Chiavacci Farley
# Chele Chiavacci Farley
# Kirsten E. Gillibrand
# Kirsten E. Gillibrand
# Kirsten E. Gillibrand
# Chele Chiavacci Farley
# Christopher S Murphy
# No other candidates

In [65]:
#There is multiple duplicate vote numbers for New York
for i in range(len(votersInfo2018)):
    if votersInfo2018['Candidate'][i] == "Kirsten E. Gillibrand":
        print(f'Gillibrand at {i}')
    elif votersInfo2018['Candidate'][i] == "Chele Chiavacci Farley":
        print(f'Farley at {i}')
    elif votersInfo2018['Candidate'][i] == "Christopher S Murphy":
        print(f'Murphey at {i}')
    

Murphey at 6
Murphey at 8
Gillibrand at 90
Farley at 91
Farley at 92
Gillibrand at 93
Gillibrand at 94
Gillibrand at 95
Farley at 96


In [66]:
#votersInfo2018.iloc[:10]

In [67]:
murphey = 37894 + 787685
votersInfo2018['Candidate_Votes'][6] = murphey
votersInfo2018 = votersInfo2018.drop(votersInfo2018.index[8])
votersInfo2018 = votersInfo2018.reset_index()
votersInfo2018 = votersInfo2018.drop(['index'], axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
#votersInfo2018.iloc[:10]

In [69]:
#votersInfo2018.iloc[87:98]

In [70]:
#Adding up Farleys votes
farley = 1730439 + 246171 + 21610
print(farley)

1998220


In [71]:
#Adding up Gillibrands votes
gillibrand = 3755489+160128+99325+41989
print(gillibrand)

4056931


In [72]:
votersInfo2018 = votersInfo2018.drop(votersInfo2018.index[91:96])
votersInfo2018 = votersInfo2018.reset_index()
votersInfo2018 = votersInfo2018.drop(['index'], axis = 1)
votersInfo2018 = votersInfo2018.drop(votersInfo2018.index[74])
votersInfo2018 = votersInfo2018.reset_index()
votersInfo2018 = votersInfo2018.drop(['index'], axis = 1)
#votersInfo2018.iloc[88:98]

In [73]:
votersInfo2018['Candidate_Votes'][89] = gillibrand
votersInfo2018['Candidate_Votes'][90] = farley

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [74]:
#votersInfo2018.iloc[88:98]

In [75]:
#votersInfo2018.iloc[60:80]

In [76]:
#votersInfo2018.iloc[:10]

In [77]:
#gillibrand + farley

# Removing Special Chars
- function removes special characters

In [78]:
def noChars(stripList):
    cleanedList = []
    for longName in stripList:
        counter = 0
        noChars = ''
        for letter in longName:
            if (letter == '.') & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == "'") & (counter < len(longName)):
                #print(longName[counter-1])
                #noChars += longName[counter-1]
                counter += 1
            elif (letter == '"') & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == ',') & (counter < len(longName)):
                counter += 1
                continue
            elif counter <= len(longName):
                noChars += letter
                counter += 1
            else:
                counter += 1
        cleanedList.append(noChars)
    return cleanedList


In [79]:
# Testing noChars
# list2 = ['Angus S. "K"ing, Jr.', '"Bob" B. B,arker', 'Write In', 'Lily C. B. Allen III', 'Amelia Airheart', "Bob O'Smirnoff"]
# clean1 = noChars(list2)
# print(clean1)

# Splitting Name
- into first, last, middle, and suffixes if possible
- need to create a list of suffixes and honorifics, ie phds, because the last grouping of strings from a name if they're an honorific are longer than 3 chars, and some names are that long or less
- have to find a way to keep names that are 3 chars or less

In [80]:
#Testing splitname first step
# splitName2018 = []
# for i in votersInfo2018['Candidate']:
#     splitName2018.append(i.split())

In [81]:
# print(len(votersInfo2018['Candidate']))
# print(len(splitName2018))

In [82]:
def splitName(column):
    splitName = []
    lastName = []
    index = []
    for i in column:
        splitName.append(i.split())
    #print(len(splitName))
    #print(splitName)
    firstName = []
    lastName = []
    middleI = []
    lastName = []
    suffix = []
    counter = 0
    for i in splitName:
        #print(len(i))
        if len(i) == 2 and (i[0].lower() != 'write'):
            firstName.append(i[0])
            lastName.append(i[1])
            index.append(counter)
            counter += 1
        elif len(i) ==3:
            firstName.append(i[0])
            middleI.append(i[1])
            lastName.append(i[2])
            index.append(counter)
            counter += 1
        elif len(i) == 4:
            firstName.append(i[0])
            middleI.append(i[1])
            suffix.append(i[-1])
            lastName.append(i[-2])
            index.append(counter)
            counter += 1
        elif (i[0] == 'Write') | (i[0] == 'Others') | (i[0] == 'scatter')|(i[0] == 'Blank Vote') |(i[0] == 'Over Vote')| (i[0] == 'None of these Candidates') |(i[0] == 'Other'):
            counter += 1
            continue
        else:
            lastName.append('twoMiddle')
            counter += 1
            index.append(counter)
    return firstName, middleI, lastName, suffix, index

In [83]:
# Testing SplitName Func
# splitNameList = splitName(clean1)
# lastName = splitNameList[2]
# print(lastName)
# splitNameList[4]

In [84]:
splitVotersInfo2014 = splitName(votersInfo2014['Candidate'])
print(len(splitVotersInfo2014[2]))
print(len(splitVotersInfo2014[4]))
lastName2014 = splitVotersInfo2014[2]
matchList = splitVotersInfo2014[4]
print(lastName2014[:20])

148
148
['Sessions', 'Begich', 'Fish', 'Gianoutsos', 'Sullivan', 'Cotton', 'Pryor', 'Swaney', 'LaFrance', 'Acosta', 'Gardner', 'Kent', 'Hammons', 'Shogan', 'Udall', 'Wade', 'Coons', 'Groff', 'Perdue', 'Nunn']


In [85]:
splitVotersInfo2016 = splitName(votersInfo2016['Candidate'])
print(len(splitVotersInfo2016[2]))
print(len(splitVotersInfo2016[4]))
lastName2016 = splitVotersInfo2016[2]
matchList = splitVotersInfo2016[4]
print(lastName2016[:20])

163
163
['Crumpton', 'Shelby', 'Miller', 'Gianoutsos', 'Metcalfe', 'Stock', 'Craig', 'Murkowski', 'Camboni', 'Kirkpatrick', 'Clark', 'Dudikoff', 'McCain', 'Swing', 'Boozman', 'Eldridge', 'Gilbert', 'Sanchez', 'Harris', 'Fiorino']


In [86]:
splitVotersInfo2018 = splitName(votersInfo2018['Candidate'])
print(len(splitVotersInfo2018[2]))
print(len(splitVotersInfo2018[4]))
lastName2018 = splitVotersInfo2018[2]
matchList = splitVotersInfo2018[4]
print(lastName2018[:20])

126
126
['McSally', 'Sinema', 'Green', 'Feinstein', 'Leon', 'Murphy', 'Corey', 'Lion', 'Russell', 'Carper', 'Arlett', 'Theodoropoulos', 'Frost', 'Scott', 'Nelson', 'Hirono', 'Curtis', 'Braun', 'Donnelly', 'Brenton']


In [87]:
#This gets the indexes of the locations that dont have write in or others
getIndex = []
howMany = []
for i in range(len(votersInfo2014)):
    if (votersInfo2014['Candidate'][i] != 'Write in') & (votersInfo2014['Candidate'][i] != 'Others') & (votersInfo2014['Candidate'][i] != 'None of these Candidates'):
        howMany.append(votersInfo2014['Candidate'][i])
        holder = votersInfo2014['Candidate'][i]
        getIndex.append(i)
        #print(f'name : {holder}, index: {i}')
print(len(howMany))
noWriteIns2014 = votersInfo2014.iloc[getIndex, :]
print(len(noWriteIns2014))
#noWriteIns

154
154


In [88]:
#This gets the indexes of the locations that dont have write in or others
getIndex = []
howMany = []
for i in range(len(votersInfo2018)):
    if (votersInfo2018['Candidate'][i] != 'Write in') & (votersInfo2018['Candidate'][i] != 'Others') & (votersInfo2018['Candidate'][i] != 'None of these Candidates'):
        howMany.append(votersInfo2018['Candidate'][i])
        holder = votersInfo2018['Candidate'][i]
        getIndex.append(i)
        #print(f'name : {holder}, index: {i}')
print(len(howMany))
noWriteIns2018 = votersInfo2018.iloc[getIndex, :]
print(len(noWriteIns2018))
#noWriteIns

126
126


In [89]:
#This gets the indexes of the locations that dont have write in or others
getIndex = []
howMany = []
for i in range(len(votersInfo2016)):
    if (votersInfo2016['Candidate'][i] != 'Write in') & (votersInfo2016['Candidate'][i] != 'Others') & (votersInfo2016['Candidate'][i] != 'None of these Candidates'):
        howMany.append(votersInfo2016['Candidate'][i])
        holder = votersInfo2016['Candidate'][i]
        getIndex.append(i)
        #print(f'name : {holder}, index: {i}')
print(len(howMany))
noWriteIns2016 = votersInfo2016.iloc[getIndex, :]
print(len(noWriteIns2016))
#noWriteIns

173
173


In [90]:
# if "McSally" in 'Linda McSally':
#     print('True')

In [91]:
#Keep this code!!!!!
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [92]:
#This code Looks for an exact match
# list1 is longer
# def exactNonMatches(longerList, shorterList):
#     nonMatches = [i for i in shorterList if i not in longerList]
#     return nonMatches

In [93]:
# smallList = ['Mary Sue', 'Parks']
# longerList = ['Lilly Anne', 'George Parks', 'Bobette Shirly', 'Mary Sue']
# print(matchLists(longerList, smallList))

In [94]:
#print(exactNonMatches(longerList, smallList))

In [95]:
longName2014 = list(noWriteIns2014['Candidate']) 
# print(len(longName))
# print(len(lastName1))
# lastName1[:10]

In [96]:
longName2016 = list(noWriteIns2016['Candidate']) 
# print(len(longName))
# print(len(lastName1))
# lastName1[:10]

In [97]:
longName2018 = list(noWriteIns2018['Candidate']) 
# print(len(longName))
# print(len(lastName1))
# lastName1[:10]

In [98]:
#longName[:10]

In [99]:
indexes2014 = matchLists(longName2014, lastName2014)
#print(indexes)

length with all: 185
length uniques only: 148


In [100]:
indexes2016 = matchLists(longName2016, lastName2016)
#print(indexes)

length with all: 204
length uniques only: 163


In [101]:
indexes2018 = matchLists(longName2018, lastName2018)
#print(indexes)

length with all: 133
length uniques only: 126


In [102]:
#Next line drops all rows that are not in the indexes list
votersLast2014 = noWriteIns2014.iloc[indexes2014, :]
votersLast2014 = votersLast2014.reset_index()
votersLast2014 = votersLast2014.drop(['index'], axis = 1)

In [103]:
#Next line drops all rows that are not in the indexes list
votersLast2016 = noWriteIns2016.iloc[indexes2016, :]
votersLast2016 = votersLast2016.reset_index()
votersLast2016 = votersLast2016.drop(['index'], axis = 1)

In [104]:
#Next line drops all rows that are not in the indexes list
votersLast2018 = noWriteIns2018.iloc[indexes2018, :]
votersLast2018 = votersLast2018.reset_index()
votersLast2018 = votersLast2018.drop(['index'], axis = 1)

In [105]:
lastName2014[:10]

['Sessions',
 'Begich',
 'Fish',
 'Gianoutsos',
 'Sullivan',
 'Cotton',
 'Pryor',
 'Swaney',
 'LaFrance',
 'Acosta']

In [106]:
lastName2016[:10]

['Crumpton',
 'Shelby',
 'Miller',
 'Gianoutsos',
 'Metcalfe',
 'Stock',
 'Craig',
 'Murkowski',
 'Camboni',
 'Kirkpatrick']

In [107]:
lastName2018[:10]

['McSally',
 'Sinema',
 'Green',
 'Feinstein',
 'Leon',
 'Murphy',
 'Corey',
 'Lion',
 'Russell',
 'Carper']

In [108]:
lastName2014 = pd.Series(lastName2014)
lastName2014[3]

'Gianoutsos'

In [109]:
lastName2016 = pd.Series(lastName2016)
lastName2016[3]

'Gianoutsos'

In [110]:
lastName2018 = pd.Series(lastName2018)
lastName2018[3]

'Feinstein'

# Isolating Last Name
- the next code adds the lastName row to the DF, and uses iloc to test if matches

In [111]:
votersLast2014.head(5)

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Alabama,AL,Jeff Sessions,Republican,795606,818090
1,Alaska,AK,Mark Begich,Democrat,129431,282400
2,Alaska,AK,Mark S. Fish,Libertarian,10512,282400
3,Alaska,AK,Ted Gianoutsos,nonaffiliated,5636,282400
4,Alaska,AK,Dan Sullivan,Republican,135445,282400


In [112]:
print(len(votersLast2014))

148


In [113]:
votersLast2014['lastName'] = lastName2014
votersLast2014.iloc[:10]

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes,lastName
0,Alabama,AL,Jeff Sessions,Republican,795606,818090,Sessions
1,Alaska,AK,Mark Begich,Democrat,129431,282400,Begich
2,Alaska,AK,Mark S. Fish,Libertarian,10512,282400,Fish
3,Alaska,AK,Ted Gianoutsos,nonaffiliated,5636,282400,Gianoutsos
4,Alaska,AK,Dan Sullivan,Republican,135445,282400,Sullivan
5,Arkansas,AR,Tom Cotton,Republican,478819,847505,Cotton
6,Arkansas,AR,Mark L. Pryor,Democrat,334174,847505,Pryor
7,Arkansas,AR,Mark H. Swaney,green,16797,847505,Swaney
8,Arkansas,AR,Nathan LaFrance,Libertarian,17210,847505,LaFrance
9,Colorado,CO,Ra_l Acosta,unaffiliated,24151,2041058,Acosta


In [114]:
print(len(votersLast2016))

163


In [115]:
votersLast2016['lastName'] = lastName2016
#votersLast2016.iloc[150:]
votersLast2016.iloc[150:200]

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes,lastName
150,Utah,UT,Stoney Fonua,independent american,27339,1115583,Fonua
151,Vermont,VT,Void Vote,No Party,466,320467,Vote
152,Vermont,VT,Scott Milne,Republican,103637,320467,Milne
153,Vermont,VT,Cris Ericson,united states marijuana,9156,320467,Ericson
154,Vermont,VT,Jerry Trudell,independent,5223,320467,Trudell
155,Vermont,VT,Pete Diamondstone,liberty union,3241,320467,Diamondstone
156,Vermont,VT,Blank Vote,No Party,6192,320467,Vote
157,Vermont,VT,Patrick J. Leahy,Democrat,192243,320467,Leahy
158,Washington,WA,Chris Vance,Republican,1329338,3243317,Vance
159,Washington,WA,Patty Murray,Democrat,1913979,3243317,Murray


In [116]:
#Test if Last Name actually Matches
votersLast2018['lastName'] = lastName2018
votersLast2018.iloc[:10]

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes,lastName
0,Arizona,AZ,Martha McSally,Republican,1135200,2384308,McSally
1,Arizona,AZ,Kyrsten Sinema,Democrat,1191100,2384308,Sinema
2,Arizona,AZ,Angela Green,green,57442,2384308,Green
3,California,CA,Dianne Feinstein,Democrat,6019422,11113364,Feinstein
4,California,CA,Kevin De Leon,Democrat,5093942,11113364,Leon
5,Connecticut,CT,Christopher S Murphy,Democrat,825579,1386840,Murphy
6,Connecticut,CT,Matthew Corey,Republican,545717,1386840,Corey
7,Connecticut,CT,Richard Lion,Libertarian,8838,1386840,Lion
8,Connecticut,CT,Jeff Russell,green,6618,1386840,Russell
9,Delaware,DE,Thomas R Carper,Democrat,217385,362592,Carper


In [117]:
matchingChairs

,Committee,C_First_Name,C_Last_Name,State_Abbrv,Chair_Names,State
0,"Agriculture, Nutrition and Forestry",Pat,Roberts,MI,Pat Roberts,Michigan
1,Appropriations,Richard,Shelby,VT,Richard Shelby,Vermont
2,Armed Services,Jim,Inhofe,RI,Jim Inhofe,Rhode Island
3,"Banking, Housing and Urban Affairs",Mike,Crapo,OH,Mike Crapo,Ohio
4,Budget,Mike,Enzi,VT,Mike Enzi,Vermont
5,"Commerce, Science and Transportation",Roger,Wicker,WA,Roger Wicker,Washington
6,Energy and Natural Resources,Lisa,Murkowski,WV,Lisa Murkowski,West Virginia
7,Environment and Public Works,John,Barrasso,DE,John Barrasso,Delaware
8,Finance,Chuck,Grassley,OR,Chuck Grassley,Oregon
9,Foreign Relations,Jim,Risch,NJ,Jim Risch,New Jersey


In [118]:
#Keep this code!!!!!
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [119]:
searchListVoters2014 = votersLast2014['lastName']
searchListVoters2016 = votersLast2016['lastName']
searchListVoters2018 = votersLast2018['lastName']
searchListChairs = matchingChairs['C_Last_Name']

In [120]:
def findString(searchString, searchList):
    i = 0
    #searchString = 'McSally'
    while i < len(searchList):
        if searchList[i] == searchString:
            print(i)
            i+= 1
        else:
            i += 1
    return i

In [121]:
for i in searchListChairs:
    findString(i, searchListVoters2014)

38
95
146
27
122
62
92
109


In [122]:
listNums2014 = [38, 95, 146, 27, 122, 62, 92, 109]
for i in listNums2014:
    print(votersLast2014['lastName'][i])

Roberts
Inhofe
Enzi
Risch
Alexander
Johnson
Johnson
Graham


In [123]:
chairs2014 = votersLast2014.iloc[listNums2014, :]
chairs2014 = chairs2014.reset_index()
chairs2014 = chairs2014.drop(['index'], axis = 1)
chairs2014

,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes,lastName
0,Kansas,KS,Pat Roberts,Republican,460350,866191,Roberts
1,Oklahoma,OK,James M. Inhofe,Republican,558166,820733,Inhofe
2,Wyoming,WY,Michael B. Enzi,Republican,121554,171153,Enzi
3,Idaho,ID,James E. Risch,Republican,285596,437170,Risch
4,Tennessee,TN,Lamar Alexander,Republican,850087,1374065,Alexander
5,Minnesota,MN,Heather Johnson,Libertarian,29685,1981528,Johnson
6,Oklahoma,OK,Connie Johnson,Democrat,237923,820890,Johnson
7,South Carolina,SC,Lindsey Graham,Republican,672941,1240075,Graham


In [124]:
contrib2014_file = "campaignContributions/contributions_2014.csv"
contrib2014 = pd.read_csv(contrib2014_file, encoding= 'unicode_escape')

FileNotFoundError: [Errno 2] No such file or directory: 'campaignContributions/contributions_2014.csv'

In [125]:
contrib2014

NameError: name 'contrib2014' is not defined

In [126]:
for i in searchListChairs:
    findString(i, searchListVoters2016)

1
47
7
56
162
93
36
63


In [ ]:
listNums2016 = [1, 47, 7, 56, 161, 93, 36, 63]
for i in listNums2016:
    print(votersLast2016['lastName'][i])

In [ ]:
chairs2016 = votersLast2016.iloc[listNums2016, :]
#chairs2016
chairs2016 = chairs2016.reset_index()
chairs2016 = chairs2016.drop(['index'], axis = 1)
chairs2016

In [ ]:
contrib2016_file = "campaignContributions/contributions_2016.csv"
contrib2016 = pd.read_csv(contrib2016_file, encoding= 'unicode_escape')

In [ ]:
contrib2016

In [ ]:
#Keep this code!!!!!
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [ ]:
contrib2016['Last_Name']

In [ ]:
for i in searchListChairs:
    findString(i, searchListVoters2018)

In [ ]:
listNums2018 = [43, 118, 72]
for i in listNums2018:
    print(votersLast2018['lastName'][i])


In [ ]:
chairs2018 = votersLast2018.iloc[listNums2018, :]
chairs2018 = chairs2018.reset_index()
chairs2018 = chairs2018.drop(['index'], axis = 1)
chairs2018

In [ ]:
contrib2018_file = "campaignContributions/contributions_2018.csv"
contrib2018 = pd.read_csv(contrib2018_file, encoding= 'unicode_escape')
contrib2018

In [ ]:
print(len(chairsAll))

In [ ]:
matchingChairs

In [ ]:
chairs2018

In [ ]:
chairs2016

In [ ]:
chairs2014

In [ ]:
chairsAll

In [ ]:
matchingChairs
matchingChairs = matchingChairs.sort_values(by='State')
matchingChairs = matchingChairs.reset_index()
matchingChairs = matchingChairs.drop(['index'], axis = 1)

In [ ]:
matchingChairs

In [ ]:
#chairsAll = chairsAll.sort_values(by='State')

In [ ]:
# len(allSenators)

In [ ]:
i = 0
while i < len(votersLast2018):
    searchString = 'McSally'
    if votersLast2018['lastName'][i] == searchString:
        print(i)
        i+= 1
    else:
        i += 1

In [ ]:
matchingChairs.iloc[:10]

In [ ]:
#Keep this code!!!!!
def matchLists(longerList, smallerList):
    index = []
    for i in smallerList:
        #print(i)
        for j in range(len(longerList)):
            if i in longerList[j]:
                #print(j)
                index.append(j)
    print(f'length with all: {len(index)}')
    index = sorted(index)
    #print(index)
    uniqueL = []
    for x in index:
        if x not in uniqueL:
            uniqueL.append(x)
    print(f'length uniques only: {len(uniqueL)}')
    return uniqueL

In [ ]:
votersMatch = votersLast['lastName']
print(len(votersMatch))
chairsMatch = matchingChairs['C_Last_Name']
print(len(chairsMatch))

In [ ]:
votersLastName = splitVotersInfo2018[2]
len(votersLastName)

In [ ]:
print(len(matchingChairs))
matchingChairs

In [ ]:
# matchingChairs['Chair_Names'] 
# votersInfo2018['Candidate']

In [ ]:
# candidates = votersInfo2018['candidate']
# totalVotes = votersInfo2018['totalvotes']
# candidateVotes = votersInfo2018['candidatevotes']
# stateAbbr = votersInfo2018['state_po']
# chairs = matchingChairs['Chair_Names']

In [ ]:
# matchList = []
# for chair in chairs:
#     for candidate in candidates:
#         if chair.lower() in candidate.lower():
#             matchList.append(chair)